In [7]:
import re
import json
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime, timedelta
import pandas_datareader as wp

pd.options.display.max_columns = None

In [8]:
# https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br
dataColumns =  pd.read_csv('statusinvest-busca-avancada.csv', sep=';',encoding='latin-1')
columns = dataColumns.columns
columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [9]:
cod = dataColumns['TICKER'].to_numpy()
len(cod)

601

In [10]:
cod

array(['AALR3', 'ABCB4', 'ABEV3', 'ADHM3', 'AERI3', 'AESB3', 'AFLT3',
       'AGRO3', 'AGXY3', 'AHEB3', 'AHEB5', 'AHEB6', 'ALLD3', 'ALPA3',
       'ALPA4', 'ALPK3', 'ALSO3', 'ALUP11', 'ALUP3', 'ALUP4', 'AMAR3',
       'AMBP3', 'AMER3', 'ANIM3', 'APER3', 'APTI3', 'APTI4', 'ARML3',
       'ARZZ3', 'ASAI3', 'ATMP3', 'ATOM3', 'AURA33', 'AURE3', 'AVLL3',
       'AZEV3', 'AZEV4', 'AZUL4', 'B3SA3', 'BAHI3', 'BALM3', 'BALM4',
       'BAUH4', 'BAZA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBML3', 'BBSE3',
       'BDLL3', 'BDLL4', 'BEEF3', 'BEES3', 'BEES4', 'BFRE11', 'BFRE12',
       'BGIP3', 'BGIP4', 'BIDI11', 'BIDI3', 'BIDI4', 'BIOM3', 'BKBR3',
       'BLAU3', 'BLUT3', 'BLUT4', 'BMEB3', 'BMEB4', 'BMGB4', 'BMIN3',
       'BMIN4', 'BMKS3', 'BMOB3', 'BNBR3', 'BOAS3', 'BOBR3', 'BOBR4',
       'BPAC11', 'BPAC3', 'BPAC5', 'BPAN4', 'BPAR3', 'BPAT33', 'BPHA3',
       'BRAP3', 'BRAP4', 'BRBI11', 'BRBI3', 'BRBI4', 'BRFS3', 'BRGE11',
       'BRGE12', 'BRGE3', 'BRGE5', 'BRGE6', 'BRGE7', 'BRGE8', 'BRIT3',
       'BR

In [11]:
urlFundamental = 'https://finance.yahoo.com/quote/{0}.SA/financials?p={1}.SA'

In [ ]:
for stock in cod:
    try:
         # 2 Return HTML
        url = urlFundamental.format(stock,stock)

        headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}
        resp = requests.get(url,headers= headers)
        print(url, resp)
        # 3 Filter page
        brute = BeautifulSoup(resp.text, "html.parser")
        pattern = re.compile(r'\s--\sData\s--\s')
        brute_datas = brute.find('script', text = pattern).contents[0]

        init = brute_datas.find("context")-2
        dados_json =json.loads(brute_datas[init:-12])

        # 4 Return Income Statement
        income_statement = dados_json['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistoryQuarterly']['incomeStatementHistory']

        financial_quarterly = []
        for s in income_statement:
            balance = {}
            for key, val in s.items():
                try:
                    balance[key] = val["raw"]
                except TypeError:
                    continue
                except KeyError:
                    continue
            financial_quarterly.append(balance)


        annual_index = []
        annual_0 = pd.DataFrame.from_dict(financial_quarterly[0], orient="index").transpose()
        annual_0 = annual_0.rename(index={0: pd.to_datetime(annual_0["endDate"], unit='s')[0]}) 
        annual_index.append(pd.to_datetime(annual_0["endDate"], unit='s')[0])
        annual_1 = pd.DataFrame.from_dict(financial_quarterly[1], orient="index").transpose()
        annual_1 = annual_1.rename(index={0:  pd.to_datetime(annual_1["endDate"], unit='s')[0]}) 
        annual_index.append(pd.to_datetime(annual_1["endDate"], unit='s')[0])
        annual_2 = pd.DataFrame.from_dict(financial_quarterly[2], orient="index").transpose()
        annual_2 = annual_2.rename(index={0: pd.to_datetime(annual_2["endDate"], unit='s')[0]}) 
        annual_index.append(pd.to_datetime(annual_2["endDate"], unit='s')[0])
        annual_3 = pd.DataFrame.from_dict(financial_quarterly[3], orient="index").transpose()
        annual_3 = annual_3.rename(index={0:  pd.to_datetime(annual_3["endDate"], unit='s')[0]}) 
        annual_index.append(pd.to_datetime(annual_3["endDate"], unit='s')[0])

        income_statement = pd.concat([annual_0, annual_1, annual_2, annual_3])

        # 5 Balance sheet
        balance_sheet = dados_json['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistoryQuarterly']['balanceSheetStatements']

        balace_sheet_quarterly = []
        for s in balance_sheet:
            balance = {}
            for key, val in s.items():
                try:
                    balance[key] = val["raw"]
                except TypeError:
                    continue
                except KeyError:
                    continue
            balace_sheet_quarterly.append(balance)

        bs_annual_0 = pd.DataFrame.from_dict(balace_sheet_quarterly[0], orient="index").transpose()
        bs_annual_0 = bs_annual_0.rename(index={0: pd.to_datetime(bs_annual_0["endDate"], unit='s')[0]}) 
        bs_annual_1 = pd.DataFrame.from_dict(balace_sheet_quarterly[1], orient="index").transpose()
        bs_annual_1 = bs_annual_1.rename(index={0: pd.to_datetime(bs_annual_1["endDate"], unit='s')[0]}) 
        bs_annual_2 = pd.DataFrame.from_dict(balace_sheet_quarterly[2], orient="index").transpose()
        bs_annual_2 = bs_annual_2.rename(index={0: pd.to_datetime(bs_annual_2["endDate"], unit='s')[0]}) 
        bs_annual_3 = pd.DataFrame.from_dict(balace_sheet_quarterly[3], orient="index").transpose()
        bs_annual_3 = bs_annual_3.rename(index={0: pd.to_datetime(bs_annual_3["endDate"], unit='s')[0]}) 

        bs_annual = pd.concat([bs_annual_0, bs_annual_1, bs_annual_2, bs_annual_3])

        ic_transpose = income_statement.transpose()
        bs_transpose = bs_annual.transpose()
        final = pd.concat([ic_transpose, bs_transpose])
        final = final.transpose()
        
        fileName = "data/{}.csv"
        fileName = fileName.format(stock)
        print(fileName)
        
        try:
            stockName = "{}.SA"
            stockName = stockName.format(stock)
            stock_0 = wp.DataReader(stockName, data_source="yahoo", start=annual_index[0] - timedelta(days=15),end=annual_index[0] - timedelta(days=10))
            stock_0 = stock_0.drop(stock_0.columns[[0,1,2,3,4]], axis = 1)
            stock_1 = wp.DataReader(stockName, data_source="yahoo", start=annual_index[1] - timedelta(days=15),end=annual_index[1] - timedelta(days=10))
            stock_1 = stock_1.drop(stock_1.columns[[0,1,2,3,4]], axis = 1)
            stock_2 = wp.DataReader(stockName, data_source="yahoo", start=annual_index[2] - timedelta(days=15),end=annual_index[2] - timedelta(days=10))
            stock_2 = stock_2.drop(stock_2.columns[[0,1,2,3,4]], axis = 1)
            stock_3 = wp.DataReader(stockName, data_source="yahoo", start=annual_index[3] - timedelta(days=15),end=annual_index[3] - timedelta(days=10))
            stock_3 = stock_3.drop(stock_3.columns[[0,1,2,3,4]], axis = 1)
        except TypeError:
            final.to_csv(fileName)
        except KeyError:
            final.to_csv(fileName)
        except IndexError:
            final.to_csv(fileName)
            
        
        try:
            amount_shares_0 = dados_json['context']['dispatcher']['stores']['QuoteTimeSeriesStore']["timeSeries"]['annualBasicAverageShares'][0]["reportedValue"]["raw"]
            final["amount_shares"] = [amount_shares_0, amount_shares_0, amount_shares_0, amount_shares_0]

            price_stock_0 = stock_0.iloc[0,0]
            price_stock_1 = stock_1.iloc[0,0]
            price_stock_2 = stock_2.iloc[0,0]
            price_stock_3 = stock_3.iloc[0,0]
            final["price_stock"] = [price_stock_0,price_stock_1,price_stock_2,price_stock_3]
        except TypeError:
            final.to_csv(fileName)
        except KeyError:
            final.to_csv(fileName)
        except IndexError:
            final.to_csv(fileName)
         
        try:
            final["LPA"] = final["netIncomeApplicableToCommonShares"]/final["amount_shares"]
            final["preco_lucro"] = final["price_stock"]/final["LPA"]
            final["P/VP"] = final["price_stock"]*final["amount_shares"]
            final["PL"] = final["totalCurrentLiabilities"]*final["amount_shares"]
            final["P/ATIVO"] = final["price_stock"]/(final["totalAssets"]/final["amount_shares"])
            final["VPA"] = final["totalStockholderEquity"]/final["amount_shares"]
            final["VM"] = final["amount_shares"]*final["price_stock"]
            final["EV"] = final["VM"]+final["shortLongTermDebt"]-final["cash"]
            final["EV"] = final["VM"]+final["shortLongTermDebt"]-final["cash"]
            final["EV/EBIT"] = final["EV"]/final["ebit"]
            final["NOPAT"] = final["ebit"]-final["incomeTaxExpense"]
            final["ROIC"] = final["NOPAT"]/final["longTermInvestments"]
            final["formula-gharant"] = (final["VPA"]*final["LPA"]*22.5)**0.5
            final["margem-seguranca"] = (final["formula-gharant"]/final["price_stock"]-1)*100
        except TypeError:
            final.to_csv(fileName)
        except KeyError:
            final.to_csv(fileName)
        except IndexError:
            final.to_csv(fileName)
            
        
    
    except TypeError:
        continue
    except KeyError:
        continue
    except IndexError:
        continue

https://finance.yahoo.com/quote/AALR3.SA/financials?p=AALR3.SA <Response [200]>
data/AALR3.csv
https://finance.yahoo.com/quote/ABCB4.SA/financials?p=ABCB4.SA <Response [200]>
data/ABCB4.csv
https://finance.yahoo.com/quote/ABEV3.SA/financials?p=ABEV3.SA <Response [200]>
data/ABEV3.csv
https://finance.yahoo.com/quote/ADHM3.SA/financials?p=ADHM3.SA <Response [200]>
data/ADHM3.csv
https://finance.yahoo.com/quote/AERI3.SA/financials?p=AERI3.SA <Response [200]>
data/AERI3.csv
https://finance.yahoo.com/quote/AESB3.SA/financials?p=AESB3.SA <Response [200]>
data/AESB3.csv
https://finance.yahoo.com/quote/AFLT3.SA/financials?p=AFLT3.SA <Response [200]>
data/AFLT3.csv
https://finance.yahoo.com/quote/AGRO3.SA/financials?p=AGRO3.SA <Response [200]>
data/AGRO3.csv
https://finance.yahoo.com/quote/AGXY3.SA/financials?p=AGXY3.SA <Response [200]>
data/AGXY3.csv
https://finance.yahoo.com/quote/AHEB3.SA/financials?p=AHEB3.SA <Response [200]>
data/AHEB3.csv
https://finance.yahoo.com/quote/AHEB5.SA/financial

In [19]:
for stock in cod:
    try:
         # 2 Return HTML
        url = urlFundamental.format(stock,stock)

        headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}
        resp = requests.get(url,headers= headers)
        print(url, resp)
        # 3 Filter page
        try:
            brute = BeautifulSoup(resp.text, "html.parser")
            pattern = re.compile(r'\s--\sData\s--\s')
            brute_datas = brute.find('script', text = pattern).contents[0]

            init = brute_datas.find("context")-2
            dados_json =json.loads(brute_datas[init:-12])

            # 4 Return Income Statement
            income_statement = dados_json['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistory']['incomeStatementHistory']
            financial_quarterly = []
            for s in income_statement:
                balance = {}
                for key, val in s.items():
                    try:
                        balance[key] = val["raw"]
                    except TypeError:
                        continue
                    except KeyError:
                        continue
                financial_quarterly.append(balance)


            annual_index = []
            annual_0 = pd.DataFrame.from_dict(financial_quarterly[0], orient="index").transpose()
            annual_0 = annual_0.rename(index={0: pd.to_datetime(annual_0["endDate"], unit='s')[0]}) 
            annual_index.append(pd.to_datetime(annual_0["endDate"], unit='s')[0])
            annual_1 = pd.DataFrame.from_dict(financial_quarterly[1], orient="index").transpose()
            annual_1 = annual_1.rename(index={0:  pd.to_datetime(annual_1["endDate"], unit='s')[0]}) 
            annual_index.append(pd.to_datetime(annual_1["endDate"], unit='s')[0])
            annual_2 = pd.DataFrame.from_dict(financial_quarterly[2], orient="index").transpose()
            annual_2 = annual_2.rename(index={0: pd.to_datetime(annual_2["endDate"], unit='s')[0]}) 
            annual_index.append(pd.to_datetime(annual_2["endDate"], unit='s')[0])
            annual_3 = pd.DataFrame.from_dict(financial_quarterly[3], orient="index").transpose()
            annual_3 = annual_3.rename(index={0:  pd.to_datetime(annual_3["endDate"], unit='s')[0]}) 
            annual_index.append(pd.to_datetime(annual_3["endDate"], unit='s')[0])

            income_statement = pd.concat([annual_0, annual_1, annual_2, annual_3])

            # 5 Balance sheet
            balance_sheet = dados_json['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistory']['balanceSheetStatements']
            balace_sheet_quarterly = []
            for s in balance_sheet:
                balance = {}
                for key, val in s.items():
                    try:
                        balance[key] = val["raw"]
                    except TypeError:
                        continue
                    except KeyError:
                        continue
                balace_sheet_quarterly.append(balance)

            bs_annual_0 = pd.DataFrame.from_dict(balace_sheet_quarterly[0], orient="index").transpose()
            bs_annual_0 = bs_annual_0.rename(index={0: pd.to_datetime(bs_annual_0["endDate"], unit='s')[0]}) 
            bs_annual_1 = pd.DataFrame.from_dict(balace_sheet_quarterly[1], orient="index").transpose()
            bs_annual_1 = bs_annual_1.rename(index={0: pd.to_datetime(bs_annual_1["endDate"], unit='s')[0]}) 
            bs_annual_2 = pd.DataFrame.from_dict(balace_sheet_quarterly[2], orient="index").transpose()
            bs_annual_2 = bs_annual_2.rename(index={0: pd.to_datetime(bs_annual_2["endDate"], unit='s')[0]}) 
            bs_annual_3 = pd.DataFrame.from_dict(balace_sheet_quarterly[3], orient="index").transpose()
            bs_annual_3 = bs_annual_3.rename(index={0: pd.to_datetime(bs_annual_3["endDate"], unit='s')[0]}) 

            bs_annual = pd.concat([bs_annual_0, bs_annual_1, bs_annual_2, bs_annual_3])

            ic_transpose = income_statement.transpose()
            bs_transpose = bs_annual.transpose()
            final = pd.concat([ic_transpose, bs_transpose])
            final = final.transpose()

            fileName = "data-annual/{}.csv"
            fileName = fileName.format(stock)
            print(fileName)
        except TypeError:
            continue
        except KeyError:
            continue
        except IndexError:
            continue
        except IndexError:
            continue
        
        try:
            stockName = "{}.SA"
            stockName = stockName.format(stock)
            stock_0 = wp.DataReader(stockName, data_source="yahoo", start=annual_index[0] - timedelta(days=15),end=annual_index[0] - timedelta(days=10))
            stock_0 = stock_0.drop(stock_0.columns[[0,1,2,3,4]], axis = 1)
            stock_1 = wp.DataReader(stockName, data_source="yahoo", start=annual_index[1] - timedelta(days=15),end=annual_index[1] - timedelta(days=10))
            stock_1 = stock_1.drop(stock_1.columns[[0,1,2,3,4]], axis = 1)
            stock_2 = wp.DataReader(stockName, data_source="yahoo", start=annual_index[2] - timedelta(days=15),end=annual_index[2] - timedelta(days=10))
            stock_2 = stock_2.drop(stock_2.columns[[0,1,2,3,4]], axis = 1)
            stock_3 = wp.DataReader(stockName, data_source="yahoo", start=annual_index[3] - timedelta(days=15),end=annual_index[3] - timedelta(days=10))
            stock_3 = stock_3.drop(stock_3.columns[[0,1,2,3,4]], axis = 1)
        except TypeError:
            final.to_csv(fileName)
        except KeyError:
            final.to_csv(fileName)
        except NoneType:
            final.to_csv(fileName)
            
        
        try:
            amount_shares_0 = dados_json['context']['dispatcher']['stores']['QuoteTimeSeriesStore']["timeSeries"]['annualBasicAverageShares'][0]["reportedValue"]["raw"]
            final["amount_shares"] = [amount_shares_0, amount_shares_0, amount_shares_0, amount_shares_0]

            price_stock_0 = stock_0.iloc[0,0]
            price_stock_1 = stock_1.iloc[0,0]
            price_stock_2 = stock_2.iloc[0,0]
            price_stock_3 = stock_3.iloc[0,0]
            final["price_stock"] = [price_stock_0,price_stock_1,price_stock_2,price_stock_3]
        except TypeError:
            final.to_csv(fileName)
        except KeyError:
            final.to_csv(fileName)
        except IndexError:
            final.to_csv(fileName)
         
        try:
            final["LPA"] = final["netIncomeApplicableToCommonShares"]/final["amount_shares"]
            final["preco_lucro"] = final["price_stock"]/final["LPA"]
            final["P/VP"] = final["price_stock"]*final["amount_shares"]
            final["PL"] = final["totalCurrentLiabilities"]*final["amount_shares"]
            final["P/ATIVO"] = final["price_stock"]/(final["totalAssets"]/final["amount_shares"])
            final["VPA"] = final["totalStockholderEquity"]/final["amount_shares"]
            final["VM"] = final["amount_shares"]*final["price_stock"]
            final["EV"] = final["VM"]+final["shortLongTermDebt"]-final["cash"]
            final["EV"] = final["VM"]+final["shortLongTermDebt"]-final["cash"]
            final["EV/EBIT"] = final["EV"]/final["ebit"]
            final["NOPAT"] = final["ebit"]-final["incomeTaxExpense"]
            final["ROIC"] = final["NOPAT"]/final["longTermInvestments"]
            final["formula-gharant"] = (final["VPA"]*final["LPA"]*22.5)**0.5
            final["margem-seguranca"] = (final["formula-gharant"]/final["price_stock"]-1)*100
        except TypeError:
            final.to_csv(fileName)
        except KeyError:
            final.to_csv(fileName)
        except IndexError:
            final.to_csv(fileName)
            
        
    
    except TypeError:
        continue
    except KeyError:
        continue
    except IndexError:
        continue

https://finance.yahoo.com/quote/RRRP3.SA/financials?p=RRRP3.SA <Response [200]>
data-annual/RRRP3.csv
https://finance.yahoo.com/quote/ALPA4.SA/financials?p=ALPA4.SA <Response [200]>
data-annual/ALPA4.csv
https://finance.yahoo.com/quote/ABEV3.SA/financials?p=ABEV3.SA <Response [200]>
data-annual/ABEV3.csv
https://finance.yahoo.com/quote/AMER3.SA/financials?p=AMER3.SA <Response [200]>
data-annual/AMER3.csv
https://finance.yahoo.com/quote/ASAI3.SA/financials?p=ASAI3.SA <Response [200]>
data-annual/ASAI3.csv
https://finance.yahoo.com/quote/AZUL4.SA/financials?p=AZUL4.SA <Response [200]>
data-annual/AZUL4.csv
https://finance.yahoo.com/quote/B3SA3.SA/financials?p=B3SA3.SA <Response [200]>
data-annual/B3SA3.csv
https://finance.yahoo.com/quote/BPAN4.SA/financials?p=BPAN4.SA <Response [200]>
data-annual/BPAN4.csv
https://finance.yahoo.com/quote/BBSE3.SA/financials?p=BBSE3.SA <Response [200]>
data-annual/BBSE3.csv
https://finance.yahoo.com/quote/BRML3.SA/financials?p=BRML3.SA <Response [200]>
da

https://finance.yahoo.com/quote/TIMS3.SA/financials?p=TIMS3.SA <Response [200]>
data-annual/TIMS3.csv
https://finance.yahoo.com/quote/TOTS3.SA/financials?p=TOTS3.SA <Response [200]>
data-annual/TOTS3.csv
https://finance.yahoo.com/quote/UGPA3.SA/financials?p=UGPA3.SA <Response [200]>
data-annual/UGPA3.csv
https://finance.yahoo.com/quote/USIM5.SA/financials?p=USIM5.SA <Response [200]>
data-annual/USIM5.csv
https://finance.yahoo.com/quote/VALE3.SA/financials?p=VALE3.SA <Response [200]>
data-annual/VALE3.csv
https://finance.yahoo.com/quote/VIIA3.SA/financials?p=VIIA3.SA <Response [200]>
data-annual/VIIA3.csv
https://finance.yahoo.com/quote/VBBR3.SA/financials?p=VBBR3.SA <Response [200]>
data-annual/VBBR3.csv
https://finance.yahoo.com/quote/WEGE3.SA/financials?p=WEGE3.SA <Response [200]>
data-annual/WEGE3.csv
https://finance.yahoo.com/quote/YDUQ3.SA/financials?p=YDUQ3.SA <Response [200]>
data-annual/YDUQ3.csv
